# Projekt z Inżynierii Uczenia Maszynowego
Semestr letni 2020, WEiTI PW
## Autorzy
Przemysław Stawczyk

Maciej Szulik


## Porównanie modeli regresorów:
- Knn
- tree regressor
- xgboost regressor

In [1]:
from sklearn.model_selection import train_test_split
import data_loader as data
import models_training as models

In [2]:
dataset = data.obtain_dataset_table()
dataset, le_cat, le_subcat, le_city = data.code_labels(dataset)

dataset

,delivery_company,city,price,category,subcategory,delivery_total_time,delivery_total_time_hours
0,360.0,1,5.126000e+03,1,2,2 days 11:00:00,59.0
1,360.0,1,6.118000e+01,0,4,2 days 11:00:00,59.0
2,620.0,1,3.274700e+02,3,0,4 days 05:00:00,101.0
3,620.0,1,5.990000e+01,0,4,4 days 05:00:00,101.0
4,620.0,1,5.897000e+01,3,9,4 days 05:00:00,101.0
...,...,...,...,...,...,...,...
6535,620.0,6,3.777000e+03,1,2,5 days 02:00:00,122.0
6536,360.0,6,7.990000e+08,1,5,3 days 13:00:00,85.0
6537,620.0,6,6.900000e+01,0,4,5 days 02:00:00,122.0
6538,360.0,6,8.999000e+01,0,4,3 days 13:00:00,85.0


In [3]:
target = dataset['delivery_total_time_hours']
data = dataset.drop(['delivery_total_time', 'delivery_total_time_hours'], axis=1)

data

,delivery_company,city,price,category,subcategory
0,360.0,1,5.126000e+03,1,2
1,360.0,1,6.118000e+01,0,4
2,620.0,1,3.274700e+02,3,0
3,620.0,1,5.990000e+01,0,4
4,620.0,1,5.897000e+01,3,9
...,...,...,...,...,...
6535,620.0,6,3.777000e+03,1,2
6536,360.0,6,7.990000e+08,1,5
6537,620.0,6,6.900000e+01,0,4
6538,360.0,6,8.999000e+01,0,4


## Split dataset on training and testing set

In [4]:
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=0)

## Train models

In [5]:
model_knn, params_knn = models.train_model(y_train, x_train, model_type='knn')
params_knn

{'weights': 'distance', 'n_neighbors': 22, 'algorithm': 'kd_tree'}

In [6]:
model_tree, params_tree = models.train_model(y_train, x_train, model_type='tree')
params_tree

{'splitter': 'random', 'random_state': 42, 'max_depth': 8, 'criterion': 'mse'}

In [7]:
model_xgb, params_xgb = models.train_model(y_train, x_train, model_type='xgb')
params_xgb


{'silent': 1,
 'seed': 42,
 'objective': 'reg:squarederror',
 'n_estimators': 600,
 'min_child_weight': 4,
 'max_depth': 7,
 'learning_rate': 0.07,
 'eval_metric': 'rmse'}

## Compare models on test data

In [8]:
predictions, error = models.compare_models([model_knn, model_tree, model_xgb], x_test, y_test)

error = error.rename(index={'mdl-1': 'kNN', 'mdl-2': 'tree', 'mdl-3': 'xgb'})
predictions = predictions.rename(columns={'mdl-1': 'kNN', 'mdl-2': 'tree', 'mdl-3': 'xgb'})

error

,max error,avg error,avg sqr error
kNN,43.025596,1.205992,2.895634e+01
tree,0.000000,0.000000,0.000000e+00
xgb,0.000355,0.000092,1.814237e-08


In [9]:
predictions.head


,delivery_company,city,price,category,subcategory,target,kNN,tree,xgb
0,360.0,3,1816.97,1,7,NaN,69.0,69.0,68.999954
1,516.0,3,2317.02,1,7,59.0,89.0,89.0,88.999878
2,620.0,1,89.90,0,4,NaN,101.0,101.0,101.000015
3,360.0,0,99.99,2,10,NaN,63.0,63.0,62.999989
4,620.0,4,89.99,0,4,101.0,97.0,97.0,96.999962
...,...,...,...,...,...,...,...,...,...
195,620.0,2,4598.00,1,2,NaN,104.0,104.0,103.999908
196,360.0,2,-249.00,3,0,NaN,58.0,58.0,58.000191
197,360.0,3,29.99,0,3,NaN,69.0,69.0,68.999954
198,620.0,4,54.99,3,0,NaN,97.0,97.0,96.999962
